In [1]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import os
import random
import ast
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from scipy import stats as st

In [2]:
spectrum_len = 500 # automate this
data_dir = os.environ['DATA_DIR']
parent_dir = os.environ['PWD']
stddata_path = os.path.join(data_dir, "StdData-" + str(spectrum_len))
plots_dir = os.path.join(data_dir, "plots")
os.chdir(os.path.join(parent_dir, "lab-notebook", "smunukutla"))

In [3]:
img = mpimg.imread(os.path.join(plots_dir, os.listdir(plots_dir)[0]))
spectrum_height = img.shape[0]
spectrum_width = img.shape[1]

In [4]:
def convertimg(img):
    newimg = np.empty([img.shape[0], img.shape[1]])
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            row = img[i][j]
            newimg[i][j] = (row[0] + row[1] + row[2])/3
    return newimg

In [5]:
data = pd.read_csv("data.csv", sep=",")
record_nums = data.iloc[0, :].tolist()
spectrum_names = data.iloc[1, :].tolist()
y = data.iloc[2, :].astype(int).tolist()
y = np.reshape(y, (len(y), 1))
num_samples = len(y)

In [6]:
spectra = np.zeros((num_samples, spectrum_height, spectrum_width))
i = 0
for num in record_nums:
    img = plt.imread(os.path.join(plots_dir, num + "-" + spectrum_names[i] + ".png")) # look into timeit, pickle file
    spectra[i] = convertimg(img)
    i += 1

In [7]:
spectra = spectra.reshape(spectra.shape[0], spectra.shape[1], spectra.shape[2], 1)
spectra.shape

(166, 288, 432, 1)

In [8]:
y_cat = to_categorical(y)

In [10]:
fi = open("indices.txt", "r")
num_runs = int(fi.readline())
num_minerals = int(fi.readline())

stats = []

for i in range(num_runs):
    train_set_indices = ast.literal_eval(fi.readline())
    test_set_indices = ast.literal_eval(fi.readline())
    dev_set_indices = ast.literal_eval(fi.readline())

    for j in train_set_indices:
        j = int(j)
    for k in test_set_indices:
        k = int(k)
    for m in dev_set_indices:
        m = int(m)
        
#     print(train_set_indices)
#     print(test_set_indices)
#     print(dev_set_indices)
    
    train_set = spectra[train_set_indices, :]
#     train_labels = y[train_set_indices, :]
    dev_set = spectra[dev_set_indices, :]
#     dev_labels = y[dev_set_indices, :]
    test_set = spectra[test_set_indices, :]
#     test_labels = y[test_set_indices, :]

#     train_labels = to_categorical(train_labels)
#     dev_labels = to_categorical(dev_labels)
#     test_labels = to_categorical(test_labels) # take apart the input and the output
    
    train_labels = y_cat[train_set_indices, :]
    dev_labels = y_cat[dev_set_indices, :]
    test_labels = y_cat[test_set_indices, :]
    
    model = Sequential()
    #add model layers
    model.add(Conv2D(32, kernel_size=10, strides=(6,6), activation='relu', input_shape=(spectra.shape[1],spectra.shape[2], 1))) # finer features at the first layer
    model.add(Conv2D(32, kernel_size=3, activation='relu')) # larger features at later layer
    model.add(Flatten())
    model.add(Dense(num_minerals, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    BATCH_SIZE = 32
    EPOCHS = 10
    
    model.fit(train_set, train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(dev_set, dev_labels))
    
    my_list = model.evaluate(test_set, test_labels, verbose=0)
    
    stats.append(my_list[1])

print("2D CNN Results:", st.describe(stats))

Train on 99 samples, validate on 33 samples
Epoch 1/10
99/99 [==============================] - 1s 9ms/sample - loss: 4.5294 - acc: 0.0707 - val_loss: 2.3178 - val_acc: 0.1515
Epoch 2/10
99/99 [==============================] - 1s 7ms/sample - loss: 2.3470 - acc: 0.2121 - val_loss: 2.1558 - val_acc: 0.3333
Epoch 3/10
99/99 [==============================] - 1s 7ms/sample - loss: 2.0383 - acc: 0.2929 - val_loss: 2.7144 - val_acc: 0.3030
Epoch 4/10
99/99 [==============================] - 1s 7ms/sample - loss: 2.1172 - acc: 0.3939 - val_loss: 2.2916 - val_acc: 0.0909
Epoch 5/10
99/99 [==============================] - 1s 7ms/sample - loss: 1.9055 - acc: 0.2323 - val_loss: 1.8244 - val_acc: 0.5152
Epoch 6/10
99/99 [==============================] - 1s 7ms/sample - loss: 1.3837 - acc: 0.9091 - val_loss: 1.6447 - val_acc: 0.4545
Epoch 7/10
99/99 [==============================] - 1s 7ms/sample - loss: 1.2965 - acc: 0.6061 - val_loss: 1.3698 - val_acc: 0.5758
Epoch 8/10
99/99 [==============

Train on 99 samples, validate on 33 samples
Epoch 1/10
99/99 [==============================] - 1s 11ms/sample - loss: 5.1716 - acc: 0.1515 - val_loss: 2.4229 - val_acc: 0.1515
Epoch 2/10
99/99 [==============================] - 1s 9ms/sample - loss: 2.5408 - acc: 0.1818 - val_loss: 2.2176 - val_acc: 0.1515
Epoch 3/10
99/99 [==============================] - 1s 9ms/sample - loss: 2.0847 - acc: 0.2020 - val_loss: 2.1896 - val_acc: 0.3939
Epoch 4/10
99/99 [==============================] - 1s 10ms/sample - loss: 1.8845 - acc: 0.5657 - val_loss: 2.1044 - val_acc: 0.3030
Epoch 5/10
99/99 [==============================] - 1s 10ms/sample - loss: 1.5839 - acc: 0.6465 - val_loss: 1.6656 - val_acc: 0.4545
Epoch 6/10
99/99 [==============================] - 1s 11ms/sample - loss: 1.3757 - acc: 0.7778 - val_loss: 1.4680 - val_acc: 0.6061
Epoch 7/10
99/99 [==============================] - 1s 10ms/sample - loss: 1.0570 - acc: 0.7980 - val_loss: 1.2828 - val_acc: 0.6364
Epoch 8/10
99/99 [=========

Train on 99 samples, validate on 33 samples
Epoch 1/10
99/99 [==============================] - 1s 10ms/sample - loss: 3.5568 - acc: 0.0909 - val_loss: 2.2501 - val_acc: 0.1515
Epoch 2/10
99/99 [==============================] - 1s 7ms/sample - loss: 2.4407 - acc: 0.1717 - val_loss: 2.3460 - val_acc: 0.1212
Epoch 3/10
99/99 [==============================] - 1s 7ms/sample - loss: 2.1810 - acc: 0.2020 - val_loss: 1.9825 - val_acc: 0.3939
Epoch 4/10
99/99 [==============================] - 1s 7ms/sample - loss: 1.8013 - acc: 0.4848 - val_loss: 2.0689 - val_acc: 0.3333
Epoch 5/10
99/99 [==============================] - 1s 7ms/sample - loss: 1.5454 - acc: 0.5455 - val_loss: 1.9789 - val_acc: 0.3939
Epoch 6/10
99/99 [==============================] - 1s 7ms/sample - loss: 1.3010 - acc: 0.6061 - val_loss: 1.2218 - val_acc: 0.6061
Epoch 7/10
99/99 [==============================] - 1s 7ms/sample - loss: 0.8034 - acc: 0.8081 - val_loss: 0.9952 - val_acc: 0.7273
Epoch 8/10
99/99 [=============

Train on 99 samples, validate on 33 samples
Epoch 1/10
99/99 [==============================] - 1s 12ms/sample - loss: 5.2034 - acc: 0.1010 - val_loss: 2.2361 - val_acc: 0.1515
Epoch 2/10
99/99 [==============================] - 1s 7ms/sample - loss: 2.2940 - acc: 0.0909 - val_loss: 2.2770 - val_acc: 0.1515
Epoch 3/10
99/99 [==============================] - 1s 7ms/sample - loss: 2.2364 - acc: 0.2222 - val_loss: 2.0389 - val_acc: 0.3030
Epoch 4/10
99/99 [==============================] - 1s 7ms/sample - loss: 1.9708 - acc: 0.3838 - val_loss: 1.9116 - val_acc: 0.3939
Epoch 5/10
99/99 [==============================] - 1s 7ms/sample - loss: 1.7774 - acc: 0.4545 - val_loss: 2.0028 - val_acc: 0.2727
Epoch 6/10
99/99 [==============================] - 1s 7ms/sample - loss: 1.5286 - acc: 0.5859 - val_loss: 1.6578 - val_acc: 0.5152
Epoch 7/10
99/99 [==============================] - 1s 7ms/sample - loss: 1.2409 - acc: 0.7778 - val_loss: 1.5248 - val_acc: 0.5152
Epoch 8/10
99/99 [=============

In [16]:
model.layers[3].output

<tf.Tensor 'dense_20/Softmax:0' shape=(?, 10) dtype=float32>